# Test accuracy of different methods on simulated data

Tested with qiime2-amplicon-2023.9

## Non-QIIME imports

In [11]:
import random
random.seed(1989)
from os import listdir
import os.path
from os.path import join
from pathlib import Path
import shutil # for unpacking zip archive
import glob # for getting just csv files from archive
import subprocess # use python variables in shell script
import zipfile # for getting zip contents
import pandas as pd
import matplotlib.pyplot as plt
import sys # stopping code
import re # regular expressions

data_dir = "../output/"
listdir(data_dir)

# simulation_archive = "../output/1000microbe_100sim_test.zip"
S01_archive = '../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100.zip'
S02_archive = '../output/S02_host_ER_BM_microbe_bm_1000_coral_tree_x100.zip'
S03_archive = '../output/S03_host_ER_BM_microbe_ou_1000_coral_tree_x100.zip'
S04_archive = '../output/S04_host_ER_BM_microbe_ou_1000_coral_tree_x100.zip'
S05_archive = '../output/S05_host_ER_BM_microbe_ou_1000_coral_tree_x100.zip'
S06_archive = '../output/S06_host_ER_BM_microbe_ou_1000_coral_tree_x100.zip'

# tmp_archive = "../output/tmp_sim_archive/"
# TODO - TMP ARCHIVE SHOULD BE INSIDE EXTRACTED ARCHIVE

## QIIME2 imports and plugins

In [12]:
import qiime2
print("QIIME2 version: " + qiime2.__version__)

from qiime2.plugins import feature_table
from qiime2 import Metadata
from qiime2.plugins.feature_table.methods import rarefy
from qiime2.plugins.diversity.pipelines import beta
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.metadata.methods import distance_matrix
from qiime2.plugins.diversity.visualizers import mantel

from math import floor

QIIME2 version: 2023.9.2
QIIME2 version: 2023.9.2


## Process universal data -- TODO: turn into function
*Input* 
- Metadata [tsv]
- Simulated feature data [tsv]

*Output*
- Metadata [Artifact]
- Feature table [Artifact]
- Distance matricies (host and microbes) [Artifact]

In [8]:
# Convert Brownian motion trait to count data

def convert_trait_to_count(trait,base=1.1,n_seqs = 10):
    scaling = 1
    new_trait = (base**trait)*n_seqs    
    new_trait *= scaling
    new_trait = new_trait.apply(floor)
    return new_trait


# Extract sub-Simulation ID [SXX_XXXX]

def extract_simulation_id(filename):
    # reg expression for getting each subsim id [SXX_XXXX]
    pattern = r'^([A-Z]+\d{2}_\d{4})'
    match = re.search(pattern, Path(filename).stem)
    if match:
        return match.group(1)
    else:
        return None

In [4]:
# # Delete tmp_archive (if it exists) before continuing to avoid inheriting old files
# tmp_archive_path_obj = Path(tmp_archive)
# if tmp_archive_path_obj.exists() and tmp_archive_path_obj.is_dir():
#     shutil.rmtree(tmp_archive)
# os.mkdir(tmp_archive)

# # Unzip simulated data archive into raw data folder
# raw_data_folder = join(tmp_archive, 'raw_data')
# shutil.unpack_archive(filename = simulation_archive, extract_dir = raw_data_folder, format = "zip")

In [9]:
def process_sim_archive(simulation_zip_archive):

    # Extract simulation zip archive into a folder of the same name
    sim_folder = os.path.splitext(simulation_zip_archive)[0]
    print("Generating processed data in: {}".format(sim_folder))
    sim_folder_path_obj = Path(sim_folder)
    if sim_folder_path_obj.exists() and sim_folder_path_obj.is_dir():
        shutil.rmtree(sim_folder)
    os.mkdir(sim_folder)
    raw_sim_data_folder = join(sim_folder, 'raw_sim_data')
    shutil.unpack_archive(filename = simulation_zip_archive, extract_dir = raw_sim_data_folder, format = "zip")
    host_files = sorted(glob.glob(join(raw_sim_data_folder, "*host_traits.tsv")))
    subsim_filestem_list = [extract_simulation_id(filename) for filename in host_files]
    feature_files = sorted(glob.glob(join(raw_sim_data_folder, "*microbial_traits.tsv")))
    n_simulations = len(host_files)


    # Create shared data folder for all processed data
    shared_data_folder = join(sim_folder, 'shared_data')
    os.mkdir(shared_data_folder)


    # Loop through simulations and generate qiime artifacts
    
    # for i in range(1): # for testing
    for i in range(n_simulations): # !final product
        
        # Produce metadata artifacts
        tmp_metadata_file = host_files[i]
        tmp_filestem = subsim_filestem_list[i]
        tmp_metadata = Metadata.load(tmp_metadata_file)
        metadata_qza_file = "".join([tmp_filestem, '_METADATA.qza'])
        tmp_metadata.save(join(shared_data_folder, metadata_qza_file))


        # Load feature table
        tmp_feature_data_file = feature_files[i]
        tmp_feature_data = pd.read_csv(tmp_feature_data_file, sep = '\t')
        tmp_feature_data = tmp_feature_data.set_index("#SampleID")
        tmp_feature_data = tmp_feature_data.apply(convert_trait_to_count)
        feature_tsv_file = "".join([tmp_filestem, '_FEATURE.tsv'])
        tmp_feature_data.transpose().to_csv(join(shared_data_folder, feature_tsv_file), sep = "\t")

        # Produce feature data artifacts

        feature_table_biom_file = "".join([tmp_filestem, '_FEATURE.json'])
        feature_table_qza_file = "".join([tmp_filestem, '_FEATURE.qza'])
        rarefied_table_qza_file = "".join([tmp_filestem, '_FEATURE_RAREFIED.qza'])
        rarefied_table_tsv_file = "".join([tmp_filestem, '_FEATURE_RAREFIED.tsv'])

        # Convert normal feature table (tsv) to BIOM format (json)
        !biom convert --input-fp {join(shared_data_folder, feature_tsv_file)} -o {join(shared_data_folder, feature_table_biom_file)} --table-type='OTU table' --to-json
        # Turn BIOM file into Qiime artifact (qza)
        !qiime tools import --input-path {join(shared_data_folder, feature_table_biom_file)} --type 'FeatureTable[Frequency]' \
            --input-format BIOMV100Format --output-path {join(shared_data_folder, feature_table_qza_file)}
        # Validate Qiime artifact file
        !qiime tools validate {join(shared_data_folder, feature_table_qza_file)}
        

        # Rarefy feature data
        feature_table = qiime2.Artifact.load(join(shared_data_folder, feature_table_qza_file))
        sequence_depth = 10000
        rarefaction_results = rarefy(feature_table, sequence_depth)
        rarefied_table = rarefaction_results.rarefied_table
        rarefied_table.save(join(shared_data_folder, rarefied_table_qza_file))
        rarefied_table.export_data(output_dir= join(shared_data_folder))
        rarefied_table_biom_file = "".join([tmp_filestem, '_FEATURE_RAREFIED.biom'])
        os.rename(join(shared_data_folder, 'feature-table.biom'), join(shared_data_folder, rarefied_table_biom_file))

        # Calculate beta diversity distance matrix
        rarefied_table = qiime2.Artifact.load(join(shared_data_folder, rarefied_table_qza_file))
        beta_div_results = beta(rarefied_table, metric = "braycurtis")
        beta_dm = beta_div_results.distance_matrix
        beta_dm_filename = "".join([tmp_filestem, "_seq_depth_", str(sequence_depth), '_FEATURE_RAREFIED_DM.qza'])
        beta_dm.save(join(shared_data_folder, beta_dm_filename))


In [10]:
process_sim_archive(S01_archive)

Generating processed data in: ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100
Generating processed data in: ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100


Imported ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0001_FEATURE.json as BIOMV100Format to ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0001_FEATURE.qza
Imported ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0001_FEATURE.json as BIOMV100Format to ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0001_FEATURE.qza


Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0001_FEATURE.qza appears to be valid at level=max.
Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0001_FEATURE.qza appears to be valid at level=max.


Imported ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0002_FEATURE.json as BIOMV100Format to ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0002_FEATURE.qza
Imported ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0002_FEATURE.json as BIOMV100Format to ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0002_FEATURE.qza


Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0002_FEATURE.qza appears to be valid at level=max.
Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0002_FEATURE.qza appears to be valid at level=max.


...

Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0098_FEATURE.qza appears to be valid at level=max.
Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0098_FEATURE.qza appears to be valid at level=max.


Imported ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0099_FEATURE.json as BIOMV100Format to ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0099_FEATURE.qza
Imported ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0099_FEATURE.json as BIOMV100Format to ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0099_FEATURE.qza


Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0099_FEATURE.qza appears to be valid at level=max.
Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0099_FEATURE.qza appears to be valid at level=max.


Imported ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0100_FEATURE.json as BIOMV100Format to ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0100_FEATURE.qza
Imported ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0100_FEATURE.json as BIOMV100Format to ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0100_FEATURE.qza


Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0100_FEATURE.qza appears to be valid at level=max.
Result ../output/S01_host_ER_BM_microbe_random_1000_coral_tree_x100/shared_data/S01_0100_FEATURE.qza appears to be valid at level=max.


## PERMANOVA Function

In [41]:
def run_PERMANOVA(simulation_zip_archive):

    sim_folder = os.path.splitext(simulation_zip_archive)[0]
    shared_data_folder = join(sim_folder, 'shared_data')
    host_files = sorted(glob.glob(join(raw_sim_data_folder, "*host_traits.tsv")))
    subsim_filestem_list = [extract_simulation_id(filename) for filename in host_files]
    feature_files = sorted(glob.glob(join(raw_sim_data_folder, "*microbial_traits.tsv")))
    n_simulations = len(host_files)

    PERMANOVA_full_results_df = pd.DataFrame(columns = ['Group 1','Group 2','Sample size','Permutations','pseudo-F','p-value','q-value'])
    
    # Create full results container
    permanova_folder_name = join(sim_folder, 'PERMANOVA_results')
    permanova_path_obj = Path(permanova_folder_name)
    if permanova_path_obj.exists() and permanova_path_obj.is_dir():
        shutil.rmtree(permanova_folder_name)
    os.mkdir(permanova_folder_name)

    # Make raw results container
    permanova_raw_results_folder = join(permanova_folder_name, 'raw_results')
    os.mkdir(permanova_raw_results_folder)

    dist_mat_qza_files = sorted(glob.glob(join(shared_data_folder, "*FEATURE_RAREFIED_DM.qza")))

    # Load metadata and run PERMANOVA
    for i in range(n_simulations): # !final product
        tmp_metadata_file = host_files[i]
        tmp_metadata = Metadata.load(tmp_metadata_file)
        discrete_trait = tmp_metadata.get_column('disc_trait_ER')
        tmp_dm = qiime2.Artifact.load(dist_mat_qza_files[i])
        permanova_results = beta_group_significance(
            distance_matrix = tmp_dm,
            metadata = discrete_trait,
            method = "permanova",
            pairwise = True
        )
        
        permanova_results_filename = join(permanova_raw_results_folder, "".join([host_filestem_list[i], '_PERMANOVA.qzv']))
        permanova_results.visualization.save(permanova_results_filename)

        # Extract permanova result from zip
        with zipfile.ZipFile(permanova_results_filename, 'r') as f:
            filelist = f.namelist()
            permanova_basepath = os.path.commonprefix(filelist)
            tmp_permanova_result_path = str(join(permanova_basepath, 'data', 'permanova-pairwise.csv'))
            f.extractall(path = permanova_raw_results_folder, members = [tmp_permanova_result_path])

        new_permanova_result_filename = "".join(['permanova_results', str(i+1), '.csv'])
        os.rename(join(permanova_raw_results_folder, permanova_basepath, 'data', 'permanova-pairwise.csv'), 
            join(permanova_raw_results_folder, new_permanova_result_filename))
        shutil.rmtree(join(permanova_raw_results_folder, permanova_basepath)) # Delete zip after extraction
        permanova_df = pd.read_csv(join(permanova_raw_results_folder, new_permanova_result_filename))
        PERMANOVA_full_results_df = pd.concat([PERMANOVA_full_results_df, permanova_df])
    
    return PERMANOVA_full_results_df

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.
The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [ ]:
permanova_results_df = run_PERMANOVA()

In [121]:
PERMANOVA_false_discovery_rate = 100*(sum(permanova_results_df['p-value'] < 0.05)/n_simulations)
print("False discovery rate: " + str(PERMANOVA_false_discovery_rate) + "%")
print("Sample size: " + str(n_simulations))
permanova_results_df

False discovery rate: 11.0%
Sample size: 100
False discovery rate: 11.0%
Sample size: 100


,Group 1,Group 2,Sample size,Permutations,pseudo-F,p-value,q-value
0,A,B,476,999,0.935217,0.428,0.428
0,A,B,476,999,1.391671,0.161,0.161
0,A,B,476,999,0.663082,0.767,0.767
0,A,B,476,999,0.646299,0.734,0.734
0,A,B,476,999,0.531741,0.889,0.889
...,...,...,...,...,...,...,...
0,A,B,476,999,0.565243,0.838,0.838
0,A,B,476,999,2.472657,0.026,0.026
0,A,B,476,999,2.786560,0.014,0.014
0,A,B,476,999,0.564782,0.866,0.866


## Mantel test function

### Functions to extract Spearman rho and p-value from html

In [10]:
# Spearman rho
def extract_spearman_rho(filename):
    # Regular expression to find spearman rho
    pattern = re.compile(r'<th>Spearman rho</th>\s*<td>(.*?)</td>', re.IGNORECASE)
    
    # Read file contents
    with open(filename, 'r') as f:
        content = f.read()
    
    # Search for pattern
    match = pattern.search(content)
    
    if match:
        # Extract the number between the <td> tags
        spearman_rho = match.group(1)
        return spearman_rho
    else:
        return None

# p-value

def extract_p_value(filename):
    # Regular expression to find p-value
    pattern = re.compile(r'<th>p-value</th>\s*<td>(.*?)</td>', re.IGNORECASE)
    
    # Read file contents
    with open(filename, 'r') as f:
        content = f.read()
    
    # Search for pattern
    match = pattern.search(content)

    if match:
        # Extract the number between the <td> tags
        p_value = match.group(1)
        return p_value
    else:
        return None

# Permutations

def extract_permutations(filename):
    # Regular expression to find p-value
    pattern = re.compile(r'<th>Permutations</th>\s*<td>(.*?)</td>', re.IGNORECASE)
    
    # Read file contents
    with open(filename, 'r') as f:
        content = f.read()
    
    # Search for pattern
    match = pattern.search(content)

    if match:
        # Extract the number between the <td> tags
        p_value = match.group(1)
        return p_value
    else:
        return None

In [125]:
example_file = '../output/tmp_sim_archive/MANTEL_results/raw_results/872fa78d-3dff-4760-a463-1e992ed3f612/data/index.html'
example_p_val = extract_p_value(example_file)
print("p-value: " + example_p_val)
example_rho = extract_spearman_rho(example_file)
print("rho: " + example_rho)
example_permutations = extract_permutations(example_file)
print("permutations: " + example_permutations)

p-value: 0.000999
rho: 0.537043
permutations: 1000
p-value: 0.000999
rho: 0.537043
permutations: 1000


### Define and run Mantel function

In [24]:
def run_mantel():
    # Create full results container
    mantel_folder_name = join(tmp_archive, 'MANTEL_results')
    mantel_path_obj = Path(mantel_folder_name)
    if mantel_path_obj.exists() and mantel_path_obj.is_dir():
        shutil.rmtree(mantel_folder_name)
    os.mkdir(mantel_folder_name)

    # Make raw results container
    mantel_raw_results_folder = join(mantel_folder_name, 'raw_results')
    os.mkdir(mantel_raw_results_folder)

    dist_mat_qza_files = sorted(glob.glob(join(shared_data_folder, "*DM.qza")))

    Mantel_full_results_df = pd.DataFrame(columns = ['Test','Permutations','p-value','Spearman rho']) # TODO Add mantel.label1 and mantel.label2

    for i in range(n_simulations): # !final product
    # for i in range(5): # testing   
        tmp_metadata_file = host_files[i]
        tmp_metadata = Metadata.load(tmp_metadata_file)
        host_cont_trait = tmp_metadata.get_column('host_cont_trait')

        host_distance_matrix_results = distance_matrix(metadata = host_cont_trait)
        host_cont_trait_dm = host_distance_matrix_results.distance_matrix
        microbiome_dm = qiime2.Artifact.load(dist_mat_qza_files[i])

        mantel_results = mantel(
            dm1 = host_cont_trait_dm,
            dm2 = microbiome_dm,
            method = "spearman",
            permutations = 10000,
            label1 = "host_cont_trait",
            label2 = "microbiome_beta_div"
        )

        mantel_results_filename = join(mantel_raw_results_folder, "".join([host_filestem_list[i], '_MANTEL.qzv']))
        mantel_results.visualization.save(mantel_results_filename)
        
        # Extract mantel result from zip
        with zipfile.ZipFile(mantel_results_filename, 'r') as f:
            filelist = f.namelist()
            mantel_basepath = os.path.commonprefix(filelist)
            tmp_mantel_result_path = str(join(mantel_basepath, 'data', 'index.html'))
            f.extractall(path = mantel_raw_results_folder, members = [tmp_mantel_result_path])
        
        new_mantel_result_filename = "".join(['mantel_result_', str(i+1), '.html'])
        os.rename(join(mantel_raw_results_folder, mantel_basepath, 'data', 'index.html'),
            join(mantel_raw_results_folder, new_mantel_result_filename))
        shutil.rmtree(join(mantel_raw_results_folder, mantel_basepath)) # Delete zip after extraction
        
        tmp_mantel_results_html = join(mantel_raw_results_folder, new_mantel_result_filename)
        tmp_permutations = int(extract_permutations(tmp_mantel_results_html))
        tmp_p_value = float(extract_p_value(tmp_mantel_results_html))
        tmp_rho = float(extract_spearman_rho(tmp_mantel_results_html))

        result_vector = ["Mantel", tmp_permutations, tmp_p_value, tmp_rho] 
        Mantel_full_results_df.loc[i,:] = result_vector
    return Mantel_full_results_df

In [25]:
mantel_results_df = run_mantel()

# TODO run with random trait values and use the same code to check if it produces significant results every time
# ^ with the same host traits

In [28]:
# print(mantel_results_df['p-value'])
# print(permanova_results_df['p-value'])
# ls = [type(item) for item in mantel_results_df['p-value']]
# print(ls)
Mantel_false_discovery_rate = 100*(sum(mantel_results_df['p-value'] < 0.05)/n_simulations)
print("False discovery rate: " + str(Mantel_false_discovery_rate) + "%")
print("Sample size: " + str(n_simulations))
mantel_results_df

False discovery rate: 100.0%
Sample size: 100
False discovery rate: 100.0%
Sample size: 100


,Test,Permutations,p-value,Spearman rho
0,Mantel,10000,0.0001,0.536122
1,Mantel,10000,0.0001,0.185757
2,Mantel,10000,0.0001,0.282656
3,Mantel,10000,0.0001,0.554965
4,Mantel,10000,0.0001,0.165143
...,...,...,...,...
95,Mantel,10000,0.0001,0.289703
96,Mantel,10000,0.0001,0.495813
97,Mantel,10000,0.0001,0.617341
98,Mantel,10000,0.0001,0.219277


In [4]:
help(feature_table.actions.rarefy)